In [1]:
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from tqdm import tqdm, trange
import pandas as pd
import pyodbc

# Set display options
pd.set_option('display.max_columns', None)  # or specify the exact number of columns you have
pd.set_option('display.max_colwidth', None)  # to show full width of the columns
pd.set_option('display.width', None)  # to auto-detect the width of your terminal
pd.set_option('display.max_rows', None)

# Overordnet information

In [108]:
df = pd.read_excel('Kommunekode.xlsx')
df.head()

,Gyldig fra,Gyldig til,Kode,Tekst
0,2007-01-01,9999-12-31 00:00:00,101,København
1,2007-01-01,9999-12-31 00:00:00,147,Frederiksberg
2,2007-01-01,9999-12-31 00:00:00,151,Ballerup
3,2007-01-01,9999-12-31 00:00:00,153,Brøndby
4,2007-01-01,9999-12-31 00:00:00,155,Dragør


In [109]:
KV_years = ['2009', '2013', '2017', '2021']
RV_years = ['2009', '2013', '2017', '2021'] 
FT_years = ['2011', '2015', '2019', '2022']
EP_years = ['2009', '2014', '2019', '2024']



In [146]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

valgsteder = []

kommuner = df['Kode'].to_list()
i=0
for kommune in tqdm(kommuner): 
    time.sleep(0.3)
    #print(kommune)
    # URL of the webpage

    # Prøv først med den første URL-formatering
    try:
        url = f'https://kmdvalg.dk/KV/2021/K84982{kommune}.htm'
        response = requests.get(url)
        response.raise_for_status()  # Tjekker for HTTP-fejl
    except requests.exceptions.HTTPError:
        try:
            # Hvis den første URL ikke virker, prøv med den alternative formatering
            url = f'https://kmdvalg.dk/KV/2021/K84979{kommune}.htm'
            response = requests.get(url)
            response.raise_for_status()  # Tjekker for HTTP-fejl
        except requests.exceptions.HTTPError:
            try:
                # Hvis den første URL ikke virker, prøv med den alternative formatering
                url = f'https://kmdvalg.dk/KV/2021/K84733{kommune}.htm'
                response = requests.get(url)
                response.raise_for_status()  # Tjekker for HTTP-fejl
            except requests.exceptions.HTTPError:
                try:
                    # Hvis den første URL ikke virker, prøv med den alternative formatering
                    url = f'https://kmdvalg.dk/KV/2021/K84713{kommune}.htm'
                    response = requests.get(url)
                    response.raise_for_status()  # Tjekker for HTTP-fejl
                except requests.exceptions.HTTPError:
                    try:
                        # Hvis den første URL ikke virker, prøv med den alternative formatering
                        url = f'https://kmdvalg.dk/KV/2021/K84712{kommune}.htm'
                        response = requests.get(url)
                        response.raise_for_status()  # Tjekker for HTTP-fejl
                    except requests.exceptions.HTTPError:
                        # Hvis begge URL'er fejler, skip til næste kommune
                        print(f"Alle URL'er fejlede for kommune {kommune}")
                        continue

    # Fetch the webpage content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the section containing the list of links
    links_section = soup.find('div', class_='list-group')

    
    for link in links_section.find_all('a', class_='list-group-item'):
        href = link['href']
        name = link.text.strip()
        kommunekode = kommune
        kommunenavn = df[df['Kode']==kommune]['Tekst'].values[0]
        
        valgsteder.append([href, name, kommunekode, kommunenavn])

    i += 1

# Convert the list to a DataFrame
df_valgsteder = pd.DataFrame(valgsteder, columns=['Link', 'Area', 'Kommunekode', 'Kommunenavn'])

# Display the DataFrame
df_valgsteder.head()


 48%|████▊     | 48/99 [00:32<00:40,  1.26it/s]

Alle URL'er fejlede for kommune 411


100%|██████████| 99/99 [01:20<00:00,  1.23it/s]


,Link,Area,Kommunekode,Kommunenavn
0,K84982101001.htm,1. Østerbro,101,København
1,K84982101002.htm,1. Nord,101,København
2,K84982101003.htm,1. Syd,101,København
3,K84982101005.htm,1. Vest,101,København
4,K84982101006.htm,1. Nordvest,101,København


In [147]:
df_valgsteder.shape

(1383, 4)

# Personlige stemmer

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Base URL for the webpage
base_url = 'https://kmdvalg.dk/KV/2021/'

# Initialize a list to collect all the data
all_data = []

# Loop through each row in df_valgsteder with tqdm for progress tracking
for index, row in tqdm(df_valgsteder.iterrows(), total=df_valgsteder.shape[0]):
    area_name = row['Area']  # Assuming the area name column is labeled 'Area'
    link = row['Link']       # Assuming the link column is labeled 'Link'

    # Modify the link to include 'B' before the .htm
    modified_link = link.replace('.htm', 'B.htm')
    
    full_url = base_url + modified_link
    response = requests.get(full_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the outer box that contains all candidate data
    outer_boxes = soup.find_all('div', class_='col-xs-12 col-sm-6 col-md-8 col-lg-8 content-block kmd-personal-votes-list')

    for box in outer_boxes:
        # Extract all rows containing candidate names and votes
        rows = box.find_all('div', class_='row table-like-row')

        for row in rows:
            name = row.find('div', class_='col-xs-7').text.strip()
            votes = row.find('div', class_='col-xs-5').text.strip()
            all_data.append([area_name, name, votes])

# Convert the collected data into a DataFrame
df_votes = pd.DataFrame(all_data, columns=['Area', 'Name', 'Votes'])

# Drop the first row
df_votes = df_votes.iloc[1:].reset_index(drop=True)


# Display the DataFrame
print(df_votes)


# Partistemmer 

In [119]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm


def partistemmer(valg, valgår):

    # Base URL for the webpage
    base_url = 'https://kmdvalg.dk/KV/'



    # Initialize a list to collect all the data
    all_data = []
    valgsteder = df_valgsteder['link'].to_list()
    kommuner = df['Kode'].to_list()
            
    for year in KV_years:

        # Loop through each row in df_valgsteder with tqdm for progress tracking
        with tqdm(total=df_valgsteder.shape[0], desc=f'Processing {kommunenavn} {year}', leave=True) as pbar_years:

            for index, row in df_valgsteder.iterrows():
                kommune = 'København'
                area_name = row['Area']  # Assuming the area name column is labeled 'Area'
                link = row['Link']       # Assuming the link column is labeled 'Link'


                # Modify the link to include 'B' before the .htm
                modified_link = link.replace('101', str(kommune))

                full_url = base_url + year + '/' + modified_link
                response = requests.get(full_url)
                soup = BeautifulSoup(response.content, 'html.parser')

                # Find the outer box that contains all candidate data
                outer_boxes = soup.find_all('div', class_='col-xs-12 col-sm-6 col-md-8 content-block kmd-parti-list')

                for box in outer_boxes:
                    # Extract all rows containing candidate names and votes
                    rows = box.find_all('div', class_='row table-like-row')

                    # Filter out the header row
                    rows = [row for row in rows if 'Kandidatlister' not in row.text]

                    # Skip the first row if it is a header
                    for row in rows:
                        
                        party_letter_span = row.find('span', class_='parti-letter')
                        if not party_letter_span:
                            continue

                        # Extract party letter, name, votes, and percentage
                        party_letter = row.find('span', class_='parti-letter').text.strip()
                        party_name = row.find('a').text.strip()
                        votes = row.find('div', class_='table-like-cell col-xs-3 col-sm-4 col-md-2 text-right roboto-bold').text.strip()
                        progress_rows = row.find_all('div', class_='row kmd-progress kmd-progress-percent')
                        percentage = progress_rows[0]['data-count-to'] if progress_rows else None
                        percentage_dif = row.find_all('div', class_='table-like-cell')[-1].text.strip()  # Extract percentage from the last relevant div

                        # Collect data into the list
                        all_data.append([kommunenavn, year, area_name, party_letter, party_name, votes, percentage, percentage_dif])

                pbar_years.update(1)  # Update the progress bar for the inner loop



        # Convert the collected data into a DataFrame
        df_votes = pd.DataFrame(all_data, columns=['Kommune', 'Valgår', 'Area', 'Party Letter', 'Party Name', 'Votes', 'Percentage', 'Percentage diff'])


        # Drop the first row
        df_votes = df_votes.reset_index(drop=True)


        # Display the DataFrame
        df_votes.head()

Processing København:   0%|          | 0/99 [00:00<?, ?it/s]

Processing Frederiksberg:   2%|▏         | 2/99 [00:32<26:31, 16.41s/it]


In [104]:
df_votes.tail(70)

,Kommune,Valgår,Area,Party Letter,Party Name,Votes,Percentage,Percentage diff
4203,København,2021,2. Øst,L,Københavnerlisten,37,0.7,"0,7%"
4204,København,2021,2. Øst,M,Danmark for Alle,28,0.5,"0,5%"
4205,København,2021,2. Øst,N,Kommunisterne,8,0.1,"0,1%"
4206,København,2021,2. Øst,O,Dansk Folkeparti,159,2.9,"2,9%"
4207,København,2021,2. Øst,P,Copenhagen,0,0.0,"0,0%"
4208,København,2021,2. Øst,Q,Feministisk Initiativ,6,0.1,"0,1%"
4209,København,2021,2. Øst,R,Kommunistisk Parti,11,0.2,"0,2%"
4210,København,2021,2. Øst,T,Frihedslisten,15,0.3,"0,3%"
4211,København,2021,2. Øst,U,Klimapartiet Momentum,40,0.7,"0,7%"
4212,København,2021,2. Øst,V,"Venstre, Danmarks Liberale Parti",390,7.0,"7,0%"


In [87]:
soup.find('div', class_='row kmd-progress kmd-progress-percent')

<div class="row kmd-progress kmd-progress-percent" data-count-to="18.9" data-count-to-digits="1">
<div class="col-xs-9">
<div class="progress">
<div aria-valuemax="100" aria-valuemin="0" aria-valuenow="18.9" class="progress-bar kommune-background-color" role="progressbar"></div>
</div>
</div>
<div class="col-xs-3 no-left-padding progress-bar-counter">
</div>
</div>

In [70]:
print(rows)

[<div class="row table-like-row"> <div class="table-like-cell col-xs-7 col-sm-6 col-md-4 roboto-bold">Kandidatlister</div> <div class="table-like-cell col-xs-3 col-sm-4 col-md-2 roboto-bold text-right">Stemmetal</div> <div class="table-like-cell no-left-padding col-xs-2 col-sm-2 col-md-1 roboto-bold text-right">(+/-)</div> <div class="table-like-cell col-xs-10 col-sm-10 col-md-4 hidden-xs hidden-sm roboto-bold">Procenter</div> <div class="table-like-cell no-left-padding col-xs-2 col-sm-2 col-md-1 hidden-xs hidden-sm roboto-bold text-right">(+/-)</div> </div>, <div class="row table-like-row">
<div class="table-like-cell col-xs-7 col-sm-6 col-md-4"><span class="parti-letter">A</span><a href="k84982101059A.htm">Socialdemokratiet</a></div>
<div class="table-like-cell col-xs-3 col-sm-4 col-md-2 text-right roboto-bold">635</div>
<div class="table-like-cell no-left-padding col-xs-2 col-sm-2 col-md-1 text-right roboto-bold vote-up">635</div>
<div class="table-like-cell col-xs-10 col-sm-10 col-